In [1]:
import sys
sys.path.append("../v3-polars")

%load_ext autoreload
%autoreload 2

from v3 import state
import numpy as np

import polars as pl

from datetime import date, timedelta, datetime, timezone
import os

from tqdm import tqdm
import matplotlib.pyplot as plt
import pandas as pd
import pickle as pkl
import json

In [2]:
WETH = ['0x82aF49447D8a07e3bd95BD0d56f35241523fBab1'.lower(), 
        '0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2'.lower(),
        '0x4200000000000000000000000000000000000006'.lower()]

In [3]:
starting = datetime(year = 2023, month = 1, day = 1)
update = False

address = '0xc31e54c7a869b9fcbecc14363cf510d1c41fa443'
arb = state.v3Pool(address, 'arbitrum', update = update)
#priceArb = arb.getPriceSeries(starting, frequency = '15m', gas = True)

address = '0x88e6a0c2ddd26feeb64f039a2c41296fcb3f5640'
eth = state.v3Pool(address, 'ethereum', update = update)
#priceEth = eth.getPriceSeries(starting, frequency = '15m', gas = True)

address = '0x85149247691df622eaf1a8bd0cafd40bc45154a9'
op = state.v3Pool(address, 'optimism', update = update)

KeyboardInterrupt: 

In [ ]:
ethFlow = pl.read_csv('data/eth_flow.csv').select('address')
ethFlow = [*ethFlow['address'].to_numpy()]

arbFlow = pl.read_csv('data/arb_flow.csv')
arbFlow = [*arbFlow['address'].to_numpy()]

ethSwaps = eth.swaps
ethArbs = ethSwaps.filter(~pl.col('to_address').is_in(ethFlow))

arbSwaps = arb.swaps
arbArbs = arbSwaps.filter(~pl.col('to_address').is_in(arbFlow))

opFlow = pl.read_csv('data/op_flow.csv')
opFlow = [*opFlow['address'].to_numpy()]

opSwaps = op.swaps
opArbs = opSwaps.filter(~pl.col('to_address').is_in(opFlow))

In [ ]:
def getRow(col, data, schema):
    return [row[schema.index(col)] for row in data]

if not os.path.exists("fees"):
    os.mkdir("fees")

starting = datetime(year = 2023, month = 6, day = 1).replace(tzinfo = timezone.utc)
ending = datetime(year = 2024, month = 1, day = 1).replace(tzinfo = timezone.utc)

for date in pl.datetime_range(starting, ending, interval = '1d', time_zone = 'UTC', eager = True):
    print(f"Starting day {date}")
    period = timedelta(days = 1)
    
    e = ethArbs.filter((pl.col('block_timestamp') < date + period) &
                        (pl.col('block_timestamp') > date))
    a = arbArbs.filter((pl.col('block_timestamp') < date + period) &
                        (pl.col('block_timestamp') > date))
    o = opArbs.filter((pl.col('block_timestamp') < date + period) &
                         (pl.col('block_timestamp') > date))
    
    for pool_name in ['eth', 'arb', 'op']:
    #for pool_name in ['op']:
        data = []
        print(f'Starting {pool_name} for {date}')

        if pool_name == 'eth':
            pool = eth
            swaps = e
        elif pool_name == 'arb':
            pool = arb
            swaps = a
        elif pool_name == 'op':
            pool = op
            swaps = o
        else:
            ValueError("Missing pool definition")
            
        doneRows = 0
        if len([f for f in os.listdir('fees') if f != ".DS_Store"]) != 0:
            doneRows = (pl.scan_parquet('fees/*.parquet')
                        .filter((pl.col('chain') == pool.chain) &
                                (pl.col('address') == pool.pool) &
                                (pl.col('ts') >= date) &
                                (pl.col('ts') <= date + period)
                               )
                        .select(pl.count())
                        .collect()
                        .item()
                        )

        if doneRows != 0:
            print(f"Found rows for {date} and {pool_name}")
            continue

        fgg0 = 0
        fgg1 = 0
        saveFees = []

        numSwaps = swaps.shape[0]
        cnt = 0
        print(f"Starting swaps of size {numSwaps} for {pool_name}")
        
        columns = ['block_timestamp', 'as_of', 'log_index', 'amount0', 'amount1', 'to_address']
        for (ts, as_of, log, amt0, amt1, toAdd) in swaps.select(columns).iter_rows():
            cnt+=1
            if cnt % 100 == 0:
                print(f'At {cnt} | pct {round(100*cnt/numSwaps, 2)}%')

            token0 = float(amt0) > 0
            
            calldata = {'as_of': as_of,
                        'tokenIn': pool.token0 if token0 else pool.token1,
                        'swapIn': amt0 if token0 else amt1,
                        'fees': True}

            amt, swapData = pool.swapIn(calldata)
            # pos amount is pool getting (the reserves are going up)
            # amount0 is change in reserves

            fees = swapData[2]
            fees_json = json.dumps(fees)
            fgg0_delta=0
            fgg1_delta=0

            for key in fees.keys():
                if calldata['tokenIn'] == pool.token0:
                    tokens, amt = fees[key]

                    fgg0_delta=(tokens / amt)
                    fgg0+=fgg0_delta

                elif calldata['tokenIn'] == pool.token1:
                    tokens, amt = fees[key]

                    fgg1_delta=(tokens / amt)
                    fgg1+=fgg1_delta
                else:
                    raise ValueError("Token not in pool")

            data.append([as_of, pool.chain, pool.pool, ts, log, toAdd, 
                         fgg0_delta, fgg1_delta, fgg0, fgg1, fees_json])

        print(f"Saving {pool_name}")

        schema = ['as_of', 'chain', 'address', 'ts', 'log', 'toAdd', 
                  'fgg0_delta', 'fgg1_delta', 'fgg0', 'fgg1', 'fees_json']

        dfData = [pl.Series('as_of', getRow('as_of', data, schema), dtype = pl.Float64),
                  pl.Series('chain', getRow('chain', data, schema), dtype = pl.Float64),
                  pl.Series('address', getRow('address', data, schema), dtype = pl.Utf8),
                    pl.Series('ts', getRow('ts', data, schema), dtype = pl.Datetime),
                    pl.Series('log', getRow('log', data, schema), dtype = pl.Int64),
                    pl.Series('toAdd', getRow('toAdd', data, schema), dtype = pl.Utf8),
                    pl.Series('fgg0_delta', getRow('fgg0_delta', data, schema), dtype = pl.Float64),
                    pl.Series('fgg1_delta', getRow('fgg1_delta', data, schema), dtype = pl.Float64),
                    pl.Series('fgg0', getRow('fgg0', data, schema), dtype = pl.Float64),
                    pl.Series('fgg1', getRow('fgg1', data, schema), dtype = pl.Float64),
                    pl.Series('fees_json', getRow('fees_json', data, schema), dtype = pl.Utf8)
                   ]

        df = pl.DataFrame(dfData)
        if not df.is_empty():
            header = len(os.listdir('fees'))
            df.write_parquet(f'fees/{pool.chain}_{header}.parquet', statistics = True, compression = 'lz4')
            print(f"Saved {pool_name}")
        else:
            print(f"Did not save {pool_name} bc size = 0")

Starting day 2023-06-01 00:00:00+00:00
Starting eth for 2023-06-01 00:00:00+00:00
Found rows for 2023-06-01 00:00:00+00:00 and eth
Starting arb for 2023-06-01 00:00:00+00:00
Found rows for 2023-06-01 00:00:00+00:00 and arb
Starting op for 2023-06-01 00:00:00+00:00
Found rows for 2023-06-01 00:00:00+00:00 and op
Starting day 2023-06-02 00:00:00+00:00
Starting eth for 2023-06-02 00:00:00+00:00
Found rows for 2023-06-02 00:00:00+00:00 and eth
Starting arb for 2023-06-02 00:00:00+00:00
Found rows for 2023-06-02 00:00:00+00:00 and arb
Starting op for 2023-06-02 00:00:00+00:00
Found rows for 2023-06-02 00:00:00+00:00 and op
Starting day 2023-06-03 00:00:00+00:00
Starting eth for 2023-06-03 00:00:00+00:00
Found rows for 2023-06-03 00:00:00+00:00 and eth
Starting arb for 2023-06-03 00:00:00+00:00
Found rows for 2023-06-03 00:00:00+00:00 and arb
Starting op for 2023-06-03 00:00:00+00:00
Found rows for 2023-06-03 00:00:00+00:00 and op
Starting day 2023-06-04 00:00:00+00:00
Starting eth for 2023-

Found rows for 2023-06-28 00:00:00+00:00 and arb
Starting op for 2023-06-28 00:00:00+00:00
Found rows for 2023-06-28 00:00:00+00:00 and op
Starting day 2023-06-29 00:00:00+00:00
Starting eth for 2023-06-29 00:00:00+00:00
Found rows for 2023-06-29 00:00:00+00:00 and eth
Starting arb for 2023-06-29 00:00:00+00:00
Found rows for 2023-06-29 00:00:00+00:00 and arb
Starting op for 2023-06-29 00:00:00+00:00
Found rows for 2023-06-29 00:00:00+00:00 and op
Starting day 2023-06-30 00:00:00+00:00
Starting eth for 2023-06-30 00:00:00+00:00
Found rows for 2023-06-30 00:00:00+00:00 and eth
Starting arb for 2023-06-30 00:00:00+00:00
Found rows for 2023-06-30 00:00:00+00:00 and arb
Starting op for 2023-06-30 00:00:00+00:00
Found rows for 2023-06-30 00:00:00+00:00 and op
Starting day 2023-07-01 00:00:00+00:00
Starting eth for 2023-07-01 00:00:00+00:00
Found rows for 2023-07-01 00:00:00+00:00 and eth
Starting arb for 2023-07-01 00:00:00+00:00
Found rows for 2023-07-01 00:00:00+00:00 and arb
Starting op 

Found rows for 2023-07-25 00:00:00+00:00 and eth
Starting arb for 2023-07-25 00:00:00+00:00
Found rows for 2023-07-25 00:00:00+00:00 and arb
Starting op for 2023-07-25 00:00:00+00:00
Found rows for 2023-07-25 00:00:00+00:00 and op
Starting day 2023-07-26 00:00:00+00:00
Starting eth for 2023-07-26 00:00:00+00:00
Found rows for 2023-07-26 00:00:00+00:00 and eth
Starting arb for 2023-07-26 00:00:00+00:00
Found rows for 2023-07-26 00:00:00+00:00 and arb
Starting op for 2023-07-26 00:00:00+00:00
Found rows for 2023-07-26 00:00:00+00:00 and op
Starting day 2023-07-27 00:00:00+00:00
Starting eth for 2023-07-27 00:00:00+00:00
Found rows for 2023-07-27 00:00:00+00:00 and eth
Starting arb for 2023-07-27 00:00:00+00:00
Found rows for 2023-07-27 00:00:00+00:00 and arb
Starting op for 2023-07-27 00:00:00+00:00
Found rows for 2023-07-27 00:00:00+00:00 and op
Starting day 2023-07-28 00:00:00+00:00
Starting eth for 2023-07-28 00:00:00+00:00
Found rows for 2023-07-28 00:00:00+00:00 and eth
Starting arb

Found rows for 2023-08-20 00:00:00+00:00 and op
Starting day 2023-08-21 00:00:00+00:00
Starting eth for 2023-08-21 00:00:00+00:00
Found rows for 2023-08-21 00:00:00+00:00 and eth
Starting arb for 2023-08-21 00:00:00+00:00
Found rows for 2023-08-21 00:00:00+00:00 and arb
Starting op for 2023-08-21 00:00:00+00:00
Found rows for 2023-08-21 00:00:00+00:00 and op
Starting day 2023-08-22 00:00:00+00:00
Starting eth for 2023-08-22 00:00:00+00:00
Found rows for 2023-08-22 00:00:00+00:00 and eth
Starting arb for 2023-08-22 00:00:00+00:00
Found rows for 2023-08-22 00:00:00+00:00 and arb
Starting op for 2023-08-22 00:00:00+00:00
Found rows for 2023-08-22 00:00:00+00:00 and op
Starting day 2023-08-23 00:00:00+00:00
Starting eth for 2023-08-23 00:00:00+00:00
Found rows for 2023-08-23 00:00:00+00:00 and eth
Starting arb for 2023-08-23 00:00:00+00:00
Found rows for 2023-08-23 00:00:00+00:00 and arb
Starting op for 2023-08-23 00:00:00+00:00
Found rows for 2023-08-23 00:00:00+00:00 and op
Starting day 

Found rows for 2023-09-16 00:00:00+00:00 and eth
Starting arb for 2023-09-16 00:00:00+00:00
Found rows for 2023-09-16 00:00:00+00:00 and arb
Starting op for 2023-09-16 00:00:00+00:00
Found rows for 2023-09-16 00:00:00+00:00 and op
Starting day 2023-09-17 00:00:00+00:00
Starting eth for 2023-09-17 00:00:00+00:00
Found rows for 2023-09-17 00:00:00+00:00 and eth
Starting arb for 2023-09-17 00:00:00+00:00
Found rows for 2023-09-17 00:00:00+00:00 and arb
Starting op for 2023-09-17 00:00:00+00:00
Found rows for 2023-09-17 00:00:00+00:00 and op
Starting day 2023-09-18 00:00:00+00:00
Starting eth for 2023-09-18 00:00:00+00:00
Found rows for 2023-09-18 00:00:00+00:00 and eth
Starting arb for 2023-09-18 00:00:00+00:00
Found rows for 2023-09-18 00:00:00+00:00 and arb
Starting op for 2023-09-18 00:00:00+00:00
Found rows for 2023-09-18 00:00:00+00:00 and op
Starting day 2023-09-19 00:00:00+00:00
Starting eth for 2023-09-19 00:00:00+00:00
Found rows for 2023-09-19 00:00:00+00:00 and eth
Starting arb

...

At 1200 | pct 10.66%
At 1300 | pct 11.55%
At 1400 | pct 12.44%
At 1500 | pct 13.33%
At 1600 | pct 14.21%
At 1700 | pct 15.1%
At 1800 | pct 15.99%
At 1900 | pct 16.88%
At 2000 | pct 17.77%
At 2100 | pct 18.66%
At 2200 | pct 19.54%
At 2300 | pct 20.43%
At 2400 | pct 21.32%
At 2500 | pct 22.21%
At 2600 | pct 23.1%
At 2700 | pct 23.99%
At 2800 | pct 24.87%
At 2900 | pct 25.76%
At 3000 | pct 26.65%
At 3100 | pct 27.54%
At 3200 | pct 28.43%
At 3300 | pct 29.32%
At 3400 | pct 30.2%
At 3500 | pct 31.09%
At 3600 | pct 31.98%
At 3700 | pct 32.87%
At 3800 | pct 33.76%
At 3900 | pct 34.65%
At 4000 | pct 35.53%
At 4100 | pct 36.42%
At 4200 | pct 37.31%
At 4300 | pct 38.2%
At 4400 | pct 39.09%
At 4500 | pct 39.98%
At 4600 | pct 40.86%
At 4700 | pct 41.75%
At 4800 | pct 42.64%
At 4900 | pct 43.53%
At 5000 | pct 44.42%
At 5100 | pct 45.31%
At 5200 | pct 46.19%
At 5300 | pct 47.08%
At 5400 | pct 47.97%
At 5500 | pct 48.86%
At 5600 | pct 49.75%
At 5700 | pct 50.64%
At 5800 | pct 51.52%
At 5900 | pct 52.

At 3400 | pct 47.8%
At 3500 | pct 49.21%
At 3600 | pct 50.61%
At 3700 | pct 52.02%
At 3800 | pct 53.42%
At 3900 | pct 54.83%
At 4000 | pct 56.24%
At 4100 | pct 57.64%
At 4200 | pct 59.05%
At 4300 | pct 60.45%
At 4400 | pct 61.86%
At 4500 | pct 63.26%
At 4600 | pct 64.67%
At 4700 | pct 66.08%
At 4800 | pct 67.48%
At 4900 | pct 68.89%
At 5000 | pct 70.29%
At 5100 | pct 71.7%
At 5200 | pct 73.11%
At 5300 | pct 74.51%
At 5400 | pct 75.92%
At 5500 | pct 77.32%
At 5600 | pct 78.73%
At 5700 | pct 80.13%
At 5800 | pct 81.54%
At 5900 | pct 82.95%
At 6000 | pct 84.35%
At 6100 | pct 85.76%
At 6200 | pct 87.16%
At 6300 | pct 88.57%
At 6400 | pct 89.98%
At 6500 | pct 91.38%
At 6600 | pct 92.79%
At 6700 | pct 94.19%
At 6800 | pct 95.6%
At 6900 | pct 97.01%
At 7000 | pct 98.41%
At 7100 | pct 99.82%
Saving arb
Saved arb
Starting op for 2023-12-09 00:00:00+00:00
Starting swaps of size 4266 for op
At 100 | pct 2.34%
At 200 | pct 4.69%
At 300 | pct 7.03%
At 400 | pct 9.38%
At 500 | pct 11.72%
At 600 | pc

At 300 | pct 4.11%
At 400 | pct 5.47%
At 500 | pct 6.84%
At 600 | pct 8.21%
At 700 | pct 9.58%
At 800 | pct 10.95%
At 900 | pct 12.32%
At 1000 | pct 13.68%
At 1100 | pct 15.05%
At 1200 | pct 16.42%
At 1300 | pct 17.79%
At 1400 | pct 19.16%
At 1500 | pct 20.53%
At 1600 | pct 21.89%
At 1700 | pct 23.26%
At 1800 | pct 24.63%
At 1900 | pct 26.0%
At 2000 | pct 27.37%
At 2100 | pct 28.74%
At 2200 | pct 30.1%
At 2300 | pct 31.47%
At 2400 | pct 32.84%
At 2500 | pct 34.21%
At 2600 | pct 35.58%
At 2700 | pct 36.95%
At 2800 | pct 38.31%
At 2900 | pct 39.68%
At 3000 | pct 41.05%
At 3100 | pct 42.42%
At 3200 | pct 43.79%
At 3300 | pct 45.16%
At 3400 | pct 46.52%
At 3500 | pct 47.89%
At 3600 | pct 49.26%
At 3700 | pct 50.63%
At 3800 | pct 52.0%
At 3900 | pct 53.37%
At 4000 | pct 54.73%
At 4100 | pct 56.1%
At 4200 | pct 57.47%
At 4300 | pct 58.84%
At 4400 | pct 60.21%
At 4500 | pct 61.58%
At 4600 | pct 62.94%
At 4700 | pct 64.31%
At 4800 | pct 65.68%
At 4900 | pct 67.05%
At 5000 | pct 68.42%
At 5100 

At 1400 | pct 31.4%
At 1500 | pct 33.65%
At 1600 | pct 35.89%
At 1700 | pct 38.13%
At 1800 | pct 40.38%
At 1900 | pct 42.62%
At 2000 | pct 44.86%
At 2100 | pct 47.11%
At 2200 | pct 49.35%
At 2300 | pct 51.59%
At 2400 | pct 53.84%
At 2500 | pct 56.08%
At 2600 | pct 58.32%
At 2700 | pct 60.57%
At 2800 | pct 62.81%
At 2900 | pct 65.05%
At 3000 | pct 67.29%
At 3100 | pct 69.54%
At 3200 | pct 71.78%
At 3300 | pct 74.02%
At 3400 | pct 76.27%
At 3500 | pct 78.51%
At 3600 | pct 80.75%
At 3700 | pct 83.0%
At 3800 | pct 85.24%
At 3900 | pct 87.48%
At 4000 | pct 89.73%
At 4100 | pct 91.97%
At 4200 | pct 94.21%
At 4300 | pct 96.46%
At 4400 | pct 98.7%
Saving op
Saved op
Starting day 2023-12-14 00:00:00+00:00
Starting eth for 2023-12-14 00:00:00+00:00
Starting swaps of size 1864 for eth
At 100 | pct 5.36%
At 200 | pct 10.73%
At 300 | pct 16.09%
At 400 | pct 21.46%
At 500 | pct 26.82%
At 600 | pct 32.19%
At 700 | pct 37.55%
At 800 | pct 42.92%
At 900 | pct 48.28%
At 1000 | pct 53.65%
At 1100 | pct 5

At 3000 | pct 65.37%
At 3100 | pct 67.55%
At 3200 | pct 69.73%
At 3300 | pct 71.91%
At 3400 | pct 74.09%
At 3500 | pct 76.27%
At 3600 | pct 78.45%
At 3700 | pct 80.63%
At 3800 | pct 82.81%
At 3900 | pct 84.99%
At 4000 | pct 87.16%
At 4100 | pct 89.34%
At 4200 | pct 91.52%
At 4300 | pct 93.7%
At 4400 | pct 95.88%
At 4500 | pct 98.06%
Saving arb
Saved arb
Starting op for 2023-12-16 00:00:00+00:00
Starting swaps of size 3802 for op
At 100 | pct 2.63%
At 200 | pct 5.26%
At 300 | pct 7.89%
At 400 | pct 10.52%
At 500 | pct 13.15%
At 600 | pct 15.78%
At 700 | pct 18.41%
At 800 | pct 21.04%
At 900 | pct 23.67%
At 1000 | pct 26.3%
At 1100 | pct 28.93%
At 1200 | pct 31.56%
At 1300 | pct 34.19%
At 1400 | pct 36.82%
At 1500 | pct 39.45%
